In [1]:
import pandas as pd

In [14]:
df=pd.read_csv(r"E:\Cyberbullying\dataset\raw\kurrek.2020.slur-corpus.csv")

In [15]:
df['body']

0               Fuck that I dont wanna watch tranny porn. 
1        Opie just wanted to have a good time. The same...
2        Fuck that faggot Fallon. Fucking sissy boy act...
3                       BbBB...b.b..b.bb but OP's a faggot
4        Who even uses the word tranny except for trans...
                               ...                        
39998    *As an alcoholic who on occasion has one too m...
39999    It's not inaccurate, it's indoctrination of yo...
40000    What did you expect?? when they banned /r/nigg...
40001    That was hilariously bad. They used the insult...
40002    Reddit in regards to a black woman taking cand...
Name: body, Length: 40003, dtype: object

In [ ]:
import pandas as pd

def load_kurrek_data():
    # Load dataset
    df = pd.read_csv(r"E:\Cyberbullying\dataset\raw\kurrek.2020.slur-corpus.csv")

    # Drop noise
    df = df[df['gold_label'] != 'CMP']

    # Map labels to binary
    label_map = {
        'DEG': '1',
        'APR': '1',
        'NDG': '0',
        'HOM': '0'
    }
    df['binary_label'] = df['gold_label'].map(label_map)

    # Optional: Drop rows with NaN (if any remain)
    df = df.dropna(subset=['binary_label', 'body'])

    return df[['body', 'binary_label']]


In [11]:
load_kurrek_data()

,body,binary_label
0,Fuck that I dont wanna watch tranny porn.,1
1,Opie just wanted to have a good time. The same...,1
2,Fuck that faggot Fallon. Fucking sissy boy act...,1
3,BbBB...b.b..b.bb but OP's a faggot,1
4,Who even uses the word tranny except for trans...,0
...,...,...
39998,*As an alcoholic who on occasion has one too m...,1
39999,"It's not inaccurate, it's indoctrination of yo...",1
40000,What did you expect?? when they banned /r/nigg...,0
40001,That was hilariously bad. They used the insult...,0
